In [ ]:
import asyncio
import os
import random

from proto_sdk import (
    Agent,
    LanguageModel,
    Message,
    TextPart,
    ToolCallPart,
    Toolkit,
    OpenAIProvider,
    AnthropicProvider,
)

import dotenv

dotenv.load_dotenv()

In [ ]:
# Set up toolkit with decorator
toolkit = Toolkit()


@toolkit.tool
def roll_dice(num_dice: int = 1, sides: int = 6) -> dict:
    """Roll one or more dice with a specified number of sides."""
    rolls = [random.randint(1, sides) for _ in range(num_dice)]
    return {"rolls": rolls, "total": sum(rolls)}


@toolkit.tool
async def get_weather(location: str, unit: str = "celsius") -> dict:
    """Get the current weather for a location."""
    # Simulate async API call
    await asyncio.sleep(0.1)
    return {"location": location, "temperature": 22, "unit": unit}

In [ ]:
api_key = os.getenv("AI_GATEWAY_API_KEY")
if not api_key:
    raise ValueError("Set AI_GATEWAY_API_KEY environment variable")

# Create agent with Anthropic provider via Vercel AI Gateway
# You can also use:
#   - AnthropicProvider(api_key=os.getenv("ANTHROPIC_API_KEY")) for direct Anthropic API
#   - OpenAIProvider(model="gpt-4o", api_key=os.getenv("OPENAI_API_KEY")) for OpenAI

provider = AnthropicProvider(
    model="anthropic/claude-sonnet-4",
    base_url="https://ai-gateway.vercel.sh",
    api_key=api_key,
    api_key_env_var="AI_GATEWAY_API_KEY",  # Stored in config for deserialization
)

agent = Agent(
    language_model=LanguageModel(provider),
    toolkit=toolkit,
)

In [ ]:
# Add user message
agent.memory.push(Message(
    role="user",
    parts=[TextPart(text="Roll 2 dice, then tell me the weather in Tokyo.")]
))

# Run agent and stream output
print("Agent output:")
print("-" * 40)

async for msg in agent.run():
    if msg.text_delta:
        print(msg.text_delta, end="", flush=True)

    if msg.is_complete:
        for p in msg.parts:
            if isinstance(p, ToolCallPart):
                print(f"\n[Tool: {p.tool_name}({p.tool_args})]")

print("\n" + "-" * 40)

In [ ]:
# Show final memory state
print("Final messages in memory:")
for m in agent.memory.messages:
    print(f"  {m.role}: {[type(p).__name__ for p in m.parts]}")

In [ ]:
# View serialized state - notice provider_name and provider_config
agent.state_dict()

In [ ]:
# View module hierarchy
agent.named_modules()

## Serialization & Deserialization

Now let's test saving and restoring agent state. This simulates persisting an agent to disk and resuming later.

In [ ]:
import json

# Serialize agent state to JSON (could be saved to disk)
state_json = json.dumps(agent.state_dict(), indent=2)
print("Serialized state:")
print(state_json)

In [ ]:
# Create a new agent with the same toolkit
# Note: Tools must be re-registered since functions can't be serialized
toolkit2 = Toolkit()

@toolkit2.tool
def roll_dice(num_dice: int = 1, sides: int = 6) -> dict:
    """Roll one or more dice with a specified number of sides."""
    rolls = [random.randint(1, sides) for _ in range(num_dice)]
    return {"rolls": rolls, "total": sum(rolls)}

@toolkit2.tool
async def get_weather(location: str, unit: str = "celsius") -> dict:
    """Get the current weather for a location."""
    await asyncio.sleep(0.1)
    return {"location": location, "temperature": 22, "unit": unit}

# Create agent2 with a dummy provider (will be replaced by load_state_dict)
agent2 = Agent(
    language_model=LanguageModel(AnthropicProvider(api_key=api_key)),
    toolkit=toolkit2,
)

print(f"agent2 memory before load: {len(agent2.memory.messages)} messages")

In [ ]:
# Load saved state into agent2
saved_state = json.loads(state_json)
agent2.load_state_dict(saved_state)

print(f"agent2 memory after load: {len(agent2.memory.messages)} messages")
print(f"agent2 provider: {agent2.llm._provider.provider_name()}")
print(f"agent2 model: {agent2.llm._provider._model}")

In [ ]:
# Show that agent2 has the conversation history
print("agent2 conversation history:")
for m in agent2.memory.messages:
    print(f"  {m.role}: {[type(p).__name__ for p in m.parts]}")

In [ ]:
# Continue the conversation with agent2!
agent2.memory.push(Message(
    role="user",
    parts=[TextPart(text="What was my dice roll total? And can you roll again?")]
))

print("Continuing conversation with agent2:")
print("-" * 40)

async for msg in agent2.run():
    if msg.text_delta:
        print(msg.text_delta, end="", flush=True)
    if msg.is_complete:
        for p in msg.parts:
            if isinstance(p, ToolCallPart):
                print(f"\n[Tool: {p.tool_name}({p.tool_args})]")

print("\n" + "-" * 40)

In [ ]:
# Verify agent2 now has more messages
print(f"agent2 final message count: {len(agent2.memory.messages)}")
print("\nFull conversation:")
for m in agent2.memory.messages:
    if m.role == "user":
        text = next((p.text for p in m.parts if isinstance(p, TextPart)), "")
        print(f"  USER: {text[:60]}..." if len(text) > 60 else f"  USER: {text}")